AP en utilisant 2 coco files pour le tracking performance
Il faut creer le fichier d'annotation apres prediction sous le bon format
Le fichier gt, doit etre bon c'est celui creer lors de la validation dataset

In [ ]:
# # Selon chat pour extraire le coco apres prediction: A ADAPTER
# outputs = model(image)  # torch tensor ou dictionnaire
# # chaque "output" contient par instance :
# # masks -> (N,H,W) booléen ou float
# # scores -> (N,)
# # labels -> (N,)
# import numpy as np
# import json
# import cv2

# predictions_coco = []

# for img_id, output in enumerate(outputs_list, start=1):
#     masks = output['masks']  # (N,H,W)
#     scores = output['scores']  # (N,)
#     labels = output['labels']  # (N,)

#     for mask, score, label in zip(masks, scores, labels):
#         # binariser le masque
#         mask_bin = (mask > 0.5).astype(np.uint8)
#         # trouver contours
#         contours, _ = cv2.findContours(mask_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#         for contour in contours:
#             contour = contour.flatten().tolist()  # COCO polygone
#             if len(contour) < 6:  # COCO exige au moins 3 points
#                 continue
#             x, y, w, h = cv2.boundingRect(mask_bin)
#             predictions_coco.append({
#                 "image_id": img_id,
#                 "category_id": int(label),
#                 "segmentation": [contour],
#                 "score": float(score),
#                 "bbox": [x, y, w, h]
#             })

# # Sauvegarder
# with open("predictions_coco.json", "w") as f:
#     json.dump(predictions_coco, f)


Il faut creer le fichier coco à partir des predictions faites qui soit compatible avec le coco du validation set.
On utilise contours_ .json et rich_ .csv

In [ ]:
# On creer le coco avec les fichier deja existant rich et contour
import json, csv

extension = 'Test1'
saveFolder = '../'
contours_json_path = saveFolder + 'contours_' + extension + '.json'
rich_csv_path = saveFolder + 'rich_' + extension + '.csv'
outputCoco_path = "predictions.json"

# Lire le fichier JSON d'entrée
with open(contours_json_path, 'r', encoding='utf-8') as f_json:
    contours = json.load(f_json)

# Lire le fichier rich.csv et ne selectionner que les colonne qui nous interressent
donnees_filtrees = []

with open(rich_csv_path, "r", encoding="utf-8") as f:
    lecteur = csv.DictReader(f)
    for ligne in lecteur:
        entree = {
            "frame": int(ligne["frame"]),
            "det_in_frame": int(ligne["det_in_frame"]),
            "x1": int(ligne["x1"]),
            "y1": int(ligne["y1"]),
            "x2": int(ligne["x2"]),
            "y2": int(ligne["y2"]),
            "score": float(ligne["score"]),
            "class_id": int(ligne["class_id"])
        }
        donnees_filtrees.append(entree)


if len(donnees_filtrees) != len(contours):
    raise("Le fichier contours et rich n'ont pas le meme nb de lignes")

outputFile = []
for idx, imgCode in enumerate(contours):
    # on verifie qu'on parle de la meme image
    i, d = map(int, imgCode.split("_"))
    if i != (donnees_filtrees[idx]["frame"]) or d != (donnees_filtrees[idx]["det_in_frame"]):
        print("WARNING: les 2 fichiers ne sont pas compatibles")
        print(i, donnees_filtrees[idx]["frame"], d, donnees_filtrees[idx]["det_in_frame"])
        continue
    
    width = (donnees_filtrees[idx]["x2"]) - (donnees_filtrees[idx]["x1"])
    heigth = (donnees_filtrees[idx]["y2"]) - (donnees_filtrees[idx]["y1"])
    prediction = {
        "image_id": donnees_filtrees[idx]["frame"],
        "category_id": donnees_filtrees[idx]["class_id"],
        "segmentation": contours[imgCode],
        "score": donnees_filtrees[idx]["score"],
        "bbox": [donnees_filtrees[idx]["x1"], donnees_filtrees[idx]["y1"], width, heigth ]
    }
    outputFile.append(prediction)
    
# Écrire dans un fichier JSON
with open(outputCoco_path, "w", encoding="utf-8") as f:
    json.dump(outputFile, f, ensure_ascii=False, indent=2)



523


In [ ]:
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import json

# --- Charger les fichiers ---
annType = 'segm'  # 'bbox' pour détection, 'segm' pour segmentation

coco_gt = COCO("annotations_gt.json")       # fichier COCO ground truth
coco_dt = coco_gt.loadRes(outputCoco_path)  # fichier prédictions

# --- Évaluation ---
coco_eval = COCOeval(coco_gt, coco_dt, annType)
coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()
